In [39]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#####################################
# Libraries
#####################################
# Common libs
import pandas as pd
import numpy as np
import sys
import os
import random
from pathlib import Path

# Image processing
import imageio
import cv2
import skimage.transform
#from skimage.transform import rescale, resize, downscale_local_mean

# Charts
import matplotlib.pyplot as plt
import seaborn as sns

# ML, statistics
import scipy
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, roc_curve, auc, roc_auc_score

# Tensorflow
#from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, RMSprop

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

INPUT_DIR = "/kaggle/input/plant-pathology-2021-fgvc8/"
PRETRAINED_DIR = "/kaggle/input/pretrained-model/"

NUM_CLASSES = 6 #12
IMAGE_SIZE= (224,224)  #(256,256)

BATCH_SIZE = 32





In [40]:
class ModelTrainer:
    """
    Create and fit the model
    """
    
    def __init__(self):
        #(18632, 2) (17700, 2) (932, 2)
        #15930 , 1770, 932
        
        self.nb_train_samples = 15930 #4275 #32706
        self.nb_validation_samples = 1770 #475 #10902
        self.batch_size=512 #512 #32 #1024   #32
               
        
        self.img_width = IMAGE_SIZE[0]
        self.img_height = IMAGE_SIZE[1]
        print(self.img_width,self.img_height)
        
    
    def create_model(self, model_name):
        if model_name == "Xception" :
                model = self.create_model_1()
        elif model_name == "XceptionDenseNet121": 
                model = self.create_model_3()
        else:
                print("Model not found")
        self.model_name = model_name
        return model
    

    #Total params: 20,886,068
    def create_model_1(self):
        pretrained_model = tf.keras.applications.xception.Xception(include_top=False, weights='imagenet', input_shape=(self.img_width, self.img_height, 3))
        model = tf.keras.models.Sequential([
            pretrained_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(NUM_CLASSES,activation='softmax')
        ])
    
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        #tf.keras.utils.plot_model(densenet_model, to_file='densenet_model.png')
        return model

    def create_model_3(self):
        path1 = PRETRAINED_DIR + 'xception_weights_tf_dim_ordering_tf_kernels_notop.h5'
        xception_model = tf.keras.models.Sequential([
           tf.keras.applications.xception.Xception(include_top=False, weights=None, input_shape=(self.img_width, self.img_height, 3)),#(512, 512, 3)),
           tf.keras.layers.GlobalAveragePooling2D(),
           #tf.keras.layers.Dense(NUM_CLASSES,activation='softmax')
           tf.keras.layers.Dense(NUM_CLASSES,activation='sigmoid')
        ])
        # Freezing the weights
        #for layer in xception_model.layers[:-1]:
        #    layer.trainable=False
            
        #xception_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        #xception_model.summary()
        path2 = PRETRAINED_DIR + 'densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5'
        densenet_model = tf.keras.models.Sequential([
            tf.keras.applications.densenet.DenseNet121(include_top=False, weights=None,input_shape=(self.img_width, self.img_height, 3)),#(512, 512, 3)),
            tf.keras.layers.GlobalAveragePooling2D(),
            #tf.keras.layers.Dense(NUM_CLASSES,activation='softmax')
            tf.keras.layers.Dense(NUM_CLASSES,activation='sigmoid')
        ])
        #for layer in densenet_model.layers[:-1]:
        #    layer.trainable=False
        #densenet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        #densenet_model.summary()
        
        inputs = tf.keras.Input(shape=(self.img_width, self.img_height, 3)) #(512, 512, 3))

        xception_output = xception_model(inputs)
        densenet_output = densenet_model(inputs)

        outputs = tf.keras.layers.average([densenet_output, xception_output])


        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        return model

In [41]:
def get_pred_label(y_pred):
    indices = []
    for pred in y_pred:
        temp = []
        for category in pred:
            #print(category)
            if category>=0.3:
                temp.append(pred.index(category))
        #print(temp)
        if temp!=[]:
            indices.append(temp)
        else:
            temp.append(np.argmax(pred))
            indices.append(temp)
    y_pred_str = []        
    for image in indices:
        temp = []
        for i in image:
            #if i > 5:
            #    temp.append(classes[0])
            #else:
            temp.append(classes[i])

        y_pred_str.append(' '.join(temp))

    #print(y_pred_str)
    return y_pred_str

classes = ['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab'] #,'frog_eye_leaf_spot complex','powdery_mildew complex', 'rust complex', 'rust frog_eye_leaf_spot', 'scab frog_eye_leaf_spot', 'scab frog_eye_leaf_spot complex']

test_files_df=pd.DataFrame()
TEST_DIR = INPUT_DIR + 'test_images/'
test_files_df['image']=os.listdir(TEST_DIR)
        
testgen = ImageDataGenerator(
            rescale=1/255.0)

test_generator = testgen.flow_from_dataframe(
    dataframe=test_files_df,
    directory=TEST_DIR,
    x_col="image",
    y_col=None,
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=False, #True,
    class_mode=None,
    target_size=IMAGE_SIZE)

Found 3 validated image filenames.


In [42]:
trainer = ModelTrainer()
model1 = trainer.create_model("XceptionDenseNet121") # 0.9062 Total params: 27,935,872
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model1.load_weights(PRETRAINED_DIR + 'best_XceptionDenseNet121_v2.h5')

model2 = trainer.create_model("XceptionDenseNet121") # 0.9062 Total params: 27,935,872
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model2.load_weights(PRETRAINED_DIR + 'best_XceptionDenseNet121_v21.h5')

model3 = trainer.create_model("XceptionDenseNet121") # 0.9062 Total params: 27,935,872
model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model3.load_weights(PRETRAINED_DIR + 'best_XceptionDenseNet121_v23.h5')

fold_var = 3 
preds = []

for i in range(fold_var):
    preds.append(model1.predict(test_generator))
    preds.append(model2.predict(test_generator))
    preds.append(model3.predict(test_generator))
    
pred = np.mean(np.array(preds), axis=0)

print(pred)
pred = pred.tolist()
testlabels = get_pred_label(pred)
print(testlabels)
        
test_files_df['labels'] = testlabels #y_pred_str
# Write to csv
test_files_df.to_csv('./submission.csv', index=False)
print("Submission completed: written submission.csv")

224 224
[[7.6000303e-01 5.1878035e-01 1.0822977e-04 5.8096820e-03 1.9374636e-03
  4.3127462e-01]
 [2.0496781e-01 8.9322662e-01 6.6006622e-05 1.2648279e-04 3.6663313e-03
  3.2507986e-02]
 [1.1687552e-02 5.0737113e-03 9.9193268e-02 8.1207855e-03 3.9434247e-03
  8.3064878e-01]]
['complex frog_eye_leaf_spot scab', 'frog_eye_leaf_spot', 'scab']
Submission completed: written submission.csv


In [43]:
# Get dataframe for submission
submitter = Submitter(model, IMAGE_SIZE)
submission_df = submitter.predict_for_submit()     
submission_df.head()

Initializing submitter
Loaded test files list
Found 3 validated image filenames.
Submission generator created
Forming submission dataframe...
[[0.8010386228561401, 0.5480356216430664, 0.00012068660726072267, 0.0009187542600557208, 0.00139617919921875, 0.42991578578948975], [0.1674143671989441, 0.9821889996528625, 0.00013410155952442437, 0.0003273215552326292, 0.0019570142030715942, 0.019639434292912483], [0.0016907602548599243, 0.0028885751962661743, 0.09923925995826721, 0.0012562423944473267, 0.0018715758342295885, 0.8805564045906067]]
[[0, 1, 5], [1], [5]]
['complex frog_eye_leaf_spot scab', 'frog_eye_leaf_spot', 'scab']
Submission completed: written submission.csv


,image,labels
0,ad8770db05586b59.jpg,complex frog_eye_leaf_spot scab
1,c7b03e718489f3ca.jpg,frog_eye_leaf_spot
2,85f8cb619c66b863.jpg,scab


In [44]:
#!cp /kaggle/input/notebook-sam/best_XceptionDenseNet121.h5 .

#import os
#print(os.listdir("/kaggle/input/"))
#os.chdir(r'/kaggle/working')
#from IPython.display import FileLink
#FileLink(r'./best_XceptionDenseNet121.h5')